In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

In [2]:
df = pd.read_csv('562_Dataset.csv')

In [3]:
county_name = list(set(df['county_name']))
county_id = {}
for i in range(len(county_name)):
    county_id[county_name[i]] = i
state_name = list(set(df['state']))
state_id = {}
for i in range(len(state_name)):
    state_id[state_name[i]] = i
region_name = list(set(df['region']))
region_id = {}
for i in range(len(region_name)):
    region_id[region_name[i]] = i
#county_id
    

In [6]:
df["county_name"].replace(county_id, inplace=True)
df["state"].replace(state_id, inplace=True)  
df["region"].replace(region_id, inplace=True) 

In [7]:
df.dropna(axis=0,inplace = True)

In [8]:
df.isnull().any()

caseid           False
race             False
vote2016         False
faminc           False
fips             False
wait             False
land_area        False
county_name      False
state            False
county_pop       False
income_county    False
region           False
dtype: bool

In [9]:
df = df[df.vote2016 != 5] #delete I don't recall candidate
df = df[df.vote2016 != 4]  # delete i did not cast a vote, meaningless
df = df[df.vote2016 != 3] #delete someone else - increases accuracy
df = df[df.wait != 6]  #delete i don't remember wait time
df = df.drop(['caseid', 'fips', 'region','county_name'], axis=1) 
#caseid is useless, fips is useless according to documentation, 
#region is redundent we have states, county_name is confusing -> different states could have same county name

In [10]:
X = df.loc[:, df.columns != 'vote2016']
y = df.loc[:,df.columns == 'vote2016']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size= 0.25, random_state = 1)

In [11]:
lm_2 = linear_model.LogisticRegression(multi_class='ovr', solver='newton-cg')
lm_2.fit(X_train, y_train)
#.         accuracy.     f1
#newton-cg: 0.61.       0.61
#sag:        0.53.      0.11
#liblinear:  0.59.      0.54
#saga:      0.52.       0.05
#default:   0.59.       0.55

/Users/katemao/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/katemao/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/katemao/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/katemao/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/katemao/opt/anaconda3/lib/python3.8/sit

LogisticRegression(multi_class='ovr', solver='newton-cg')

In [12]:
lm_2.score(X_test, y_test)

0.6055443704506737

In [13]:
print(metrics.classification_report(y_test, lm_2.predict(X_test)))

              precision    recall  f1-score   support

         1.0       0.58      0.64      0.61      3107
         2.0       0.63      0.57      0.60      3350

    accuracy                           0.61      6457
   macro avg       0.61      0.61      0.61      6457
weighted avg       0.61      0.61      0.61      6457

